In [1]:
import pandas as pd
# https://www.geeksforgeeks.org/bag-of-word-and-frequency-count-in-text-using-sklearn/
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
impression_raw = pd.read_csv("Final_Impressions.csv")
labels_raw = pd.read_csv("Final_Impressions_labels.csv")

In [3]:
labels_df_1 = labels_raw[['impression_id','pe_acute', 'pe_subsegmentalonly', 'pe_positive']]

In [4]:
# need to obtain all the data that has valid impressions need to remove unusable data
merged_data = pd.merge(impression_raw, labels_df_1, on ="impression_id")
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23265 entries, 0 to 23264
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   impression_id        23265 non-null  object
 1   impressions          23265 non-null  object
 2   pe_acute             23265 non-null  int64 
 3   pe_subsegmentalonly  23265 non-null  int64 
 4   pe_positive          23265 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 908.9+ KB


In [5]:
vectorizer = TfidfVectorizer(lowercase = True, stop_words = "english", max_features = 1000)
x = vectorizer.fit_transform(merged_data['impressions'])
y = merged_data[['pe_acute', 'pe_subsegmentalonly', 'pe_positive']]

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 0)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

model = OneVsRestClassifier(LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)

y_preds = model.predict(X_test)

model.score(X_test, y_test)

0.9133574007220217

In [11]:
from sklearn.tree import DecisionTreeClassifier
model_2 = OneVsRestClassifier(DecisionTreeClassifier(max_depth=6))
model_2.fit(X_train, y_train)
model_2.score(X_test, y_test)

0.8920405707409318

In [8]:
from sklearn.metrics  import classification_report
print("Classification Report:\n")
print(classification_report(y_test, y_preds, target_names=['pe_acute', 'pe_subsegmentalonly', 'pe_positive']))

Classification Report:

                     precision    recall  f1-score   support

           pe_acute       0.91      0.81      0.85      1035
pe_subsegmentalonly       0.74      0.31      0.44       247
        pe_positive       0.92      0.83      0.87      1164

          micro avg       0.91      0.77      0.83      2446
          macro avg       0.86      0.65      0.72      2446
       weighted avg       0.90      0.77      0.82      2446
        samples avg       0.16      0.15      0.16      2446



/home/anikeet/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/anikeet/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/anikeet/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
from sklearn.metrics import multilabel_confusion_matrix

conf_matrix = multilabel_confusion_matrix(y_test, y_preds)

In [14]:
conf_matrix

# result - Most of the data was classified correct with more things being classified as false negatives

array([[[4700,   82],
        [ 201,  834]],

       [[5543,   27],
        [ 170,   77]],

       [[4567,   86],
        [ 194,  970]]])

In [13]:
non_zero_data = merged_data[(merged_data[['pe_acute', 'pe_subsegmentalonly', 'pe_positive']].sum(axis=1)) > 0]

In [14]:
# testing data with all of non-zero outputs
X_non_zero = vectorizer.transform(non_zero_data["impressions"])
y_preds_non_zero = non_zero_data[['pe_acute', 'pe_subsegmentalonly', 'pe_positive']]

model.score(X_non_zero, y_preds_non_zero)

0.6551797489895766